In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
#     "unsloth/llama-2-7b-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit",
#     "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
#     "unsloth/gemma-2b-bnb-4bit",
#     "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
#     "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Data preprocessing

In [ ]:
alpaca_prompt = """Below is an instruction that provides a dialogue, paired with a summary of the dialogue that sums up the important or relevant information of the dialogue precisely.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = "Summarize the given input"
    inputs       = examples["dialogue"]
    outputs      = examples["summary"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


In [ ]:

from datasets import load_dataset


import pandas as pd

train_data = pd.read_csv('samsum-train.csv')
test_data = pd.read_csv('samsum-test.csv')
val_data = pd.read_csv('samsum-validation.csv')

# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
mask = train_data['dialogue'].isnull() # Creating mask with null dialogues
filtered_train = train_data[mask] # filtering dataframe
filtered_train # Visualizing

train_data = train_data.dropna()

In [ ]:
import re
def clean_tags(text):
    clean = re.compile('<.*?>') # Compiling tags
    clean = re.sub(clean, '', text) # Replacing tags text by an empty string

    # Removing empty dialogues
    clean = '\n'.join([line for line in clean.split('\n') if not re.match('.*:\s*$', line)])

    return clean


# Defining function to clean every text in the dataset.
def clean_df(df, cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(clean_tags)
    return df

In [ ]:
# Cleaning texts in all datasets
train = clean_df(train_data,['dialogue', 'summary'])
test = clean_df(test_data,['dialogue', 'summary'])
val = clean_df(val_data,['dialogue', 'summary'])

In [ ]:
from datasets import Dataset

# Transforming dataframes into datasets
train_ds = Dataset.from_pandas(train)
test_ds = Dataset.from_pandas(test)
val_ds = Dataset.from_pandas(val)

# Visualizing results
print(train_ds)
print('\n' * 2)
print(test_ds)
print('\n' * 2)
print(val_ds)

Dataset({
    features: ['id', 'dialogue', 'summary', '__index_level_0__'],
    num_rows: 14731
})



Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 819
})



Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 818
})


In [ ]:
#dataset = dataset.map(formatting_prompts_func, batched = True,)
# Applying preprocess_function to the datasets
tokenized_train = train_ds.map(formatting_prompts_func, batched=True,
                               remove_columns=['id', 'dialogue', 'summary', '__index_level_0__']) # Removing features

tokenized_test = test_ds.map(formatting_prompts_func, batched=True,
                               remove_columns=['id', 'dialogue', 'summary']) # Removing features

tokenized_val = val_ds.map(formatting_prompts_func, batched=True,
                               remove_columns=['id', 'dialogue', 'summary']) # Removing features

# Printing results
print('\n' * 3)
print('Preprocessed Training Dataset:\n')
print(tokenized_train)
print('\n' * 2)
print('Preprocessed Test Dataset:\n')
print(tokenized_test)
print('\n' * 2)
print('Preprocessed Validation Dataset:\n')
print(tokenized_val)

Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]





Preprocessed Training Dataset:

Dataset({
    features: ['text'],
    num_rows: 375
})



Preprocessed Test Dataset:

Dataset({
    features: ['text'],
    num_rows: 25
})



Preprocessed Validation Dataset:

Dataset({
    features: ['text'],
    num_rows: 25
})


Train the model


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenized_train,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/375 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/25 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.605 GB of memory reserved.


In [ ]:
import wandb
wandb.init(mode="disabled")

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 375 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.477600
2,2.941800
3,3.427600
4,3.133200
5,3.003100
6,3.246000
7,3.038300
8,3.017700
9,3.252900
10,2.687100


inference

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Summarize the dialogue provided", # instruction
        "Hannah: Hey, do you have Betty's number?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that provides a dialogue, paired with a summary of the dialogue that sums up the important or relevant information of the dialogue precisely.\n\n### Instruction:\nSummarize the dialogue provided\n\n### Input:\nHannah: Hey, do you have Betty's number?\n\n### Response:\nHannah wants Betty's number. <|end_of_text|>"]

using text streamer for continuous inference

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Summarize the dialogue provided", # instruction
        "Hannah: Hey, do you have Betty's number?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that provides a dialogue, paired with a summary of the dialogue that sums up the important or relevant information of the dialogue precisely.

### Instruction:
Summarize the dialogue provided

### Input:
Hannah: Hey, do you have Betty's number?

### Response:
Hannah wants Betty's number.<|end_of_text|>


In [ ]:
from transformers import TextStreamer
from torch.utils.data import DataLoader
import json

alpaca_prompt = """Below is an instruction that provides a dialogue, paired with a summary of the dialogue that sums up the important or relevant information of the dialogue precisely.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Define a DataLoader for tokenized_test
batch_size = 8  # Adjust based on your GPU memory
test_loader = DataLoader(tokenized_test, batch_size=batch_size)

results = []
model.eval()


text_streamer = TextStreamer(tokenizer)


with torch.no_grad():
    for batch in test_loader:
        # Keep only relevant keys
        inputs = {
            key: value.to("cuda")
            for key, value in batch.items()
            if key in ["input_ids", "attention_mask"]
        }
        outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)
        decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        print(decoded_outputs)
        results.extend(decoded_outputs)


# # Save or print results
# with open("results.json", "w") as f:
#     json.dump(results, f)

# for result in results:
#     print(result)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
e

### Input:
Sam: Hi, what's up?
Miguel: Nothing much, how about you?
Sam: Not much, just working on a paper.
Miguel: I was just thinking about you, have you seen the new film yet?
Sam: No, I haven't, but I'm looking forward to it.
Miguel: Me too, it looks really good.
Sam: Yeah, I'm sure it will be a hit.
Miguel
["Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\ne\n\n### Input:\nSam: Hi, what's up?\r\nMiguel: Nothing much, how about you?\r\nSam: Not much, just working on a paper.\r\nMiguel: I was just thinking about you, have you seen the new film yet?\r\nSam: No, I haven't, but I'm looking forward to it.\r\nMiguel: Me too, it looks really good.\r\nSam: Yeah, I'm sur

In [ ]:

#this only saves LoRa adapters
model.save_pretrained("lora_model") # Local saving


In [ ]:
#if want to load LoRA adapters set False to True

if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Summarize the dialogue provided", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that provides a dialogue, paired with a summary of the dialogue that sums up the important or relevant information of the dialogue precisely.\n\n### Instruction:\nSummarize the dialogue provided\n\n### Input:\n\n\n### Response:\nMegan: Hello, I'm Megan, I'm calling from the office of the president.\r\nMegan: I'd like to invite you to the inauguration of the new president.\r\nMegan: It's going to be tomorrow at 10 am in the city hall.\r\nMegan: Could you please come?\r\nMegan"]